# Experiments

In [ ]:
#!pip install tensorflow==2.10.0
#!pip install docplex
#!pip install cplex

In [1]:
import pandas as pd
import tensorflow as tf

from milp import codify_network
from teste import get_minimal_explanation

# For type annotations
import numpy as np

2023-06-14 16:54:38.075630: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 16:54:38.214456: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-14 16:54:38.214469: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-14 16:54:38.237754: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-14 16:54:39.200060: W tensorflow/stream_executor/platform/de

In [2]:
dataset_name = 'glass'

training_data = pd.read_csv(f'datasets/{dataset_name}/train.csv')
testing_data = pd.read_csv(f'datasets/{dataset_name}/test.csv')

dataframe = pd.concat([training_data, testing_data])

keras_model = tf.keras.models.load_model(f'datasets/{dataset_name}/model_2layers_{dataset_name}.h5')

data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

2023-06-14 16:54:40.454857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 16:54:40.455087: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-14 16:54:40.455147: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-06-14 16:54:40.455198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-06-14 16:54:40.455249: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [3]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

In [4]:
#print(mp_model.export_as_lp_string())

In [5]:
# i = 134 is also a nice value to study
i = 138
print('i =', i)
network_input = data[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification: np.int64 = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 138
1/1 [==============================] - 0s 13ms/step
Predictions: (ndarray[ndarray[<class 'numpy.float32'>]]) 0.0007575714
Network output: (<class 'numpy.int64'>) 1


In [6]:
mdl_aux = mp_model.clone()
minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, n_classes, 'fischetti', output_bounds)
minimal_explanation

[docplex.mp.LinearConstraint[input1](x_0,EQ,2.967691214515491),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-1.408120229258977),
 docplex.mp.LinearConstraint[input6](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input7](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

### Improving the Explanation

In [7]:
import docplex

In [157]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

In [158]:
#print(testing_model.export_as_lp_string())

In [159]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input1](x_0,EQ,2.967691214515491),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-1.408120229258977),
 docplex.mp.LinearConstraint[input6](x_5,EQ,-0.790702170757714),
 docplex.mp.LinearConstraint[input7](x_6,EQ,4.24127975754059),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

In [160]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

In [161]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,2.967691214515491),
 docplex.mp.LinearConstraint[input GE](x_0,GE,2.967691214515491),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.408120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.790702170757714),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.24127975754059),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.24127975754059),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.6037614142464092),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [162]:
#print(testing_model.export_as_lp_string())

In [163]:
def log_and_improve_explanation(minimal_explanation: list, epsilon: float):
	for constraint in minimal_explanation:
		testing_model.solve()
		print('Initial constraint:' + '\t', constraint)

		variable = constraint.lhs
		while testing_model.solution is None:
			if constraint.sense == docplex.mp.constants.ComparisonType.LE:
				if constraint.rhs.constant <= variable.ub:
					constraint.rhs += epsilon
				else:
					break
			elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
				if constraint.rhs.constant >= variable.lb:
					constraint.rhs -= epsilon
				else:
					break
			else:
				raise Exception('Constraint sense was neither LE nor GE')

			testing_model.solve()

		# Undo last operation
		if constraint.sense == docplex.mp.constants.ComparisonType.LE:
			constraint.rhs -= epsilon
		elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
			constraint.rhs += epsilon

		print('Final constraint:' + '\t', constraint)
		print()

In [164]:
def find_bounds(minimal_explanation: list):
    for constraint in minimal_explanation:
        #testing_model.solve()
        print('Initial constraint:' + '\t', constraint)

        variable = constraint.lhs
        print(f"variable {variable} ub {variable.ub}")
        print(f"variable {variable} lb {variable.lb}")

def find_constraints(minimal_explanation: list):
    for constraint in minimal_explanation:
        print(constraint)
        if constraint.sense == docplex.mp.constants.ComparisonType.GE:
            print('\n')

In [165]:
'''
for constraint in linear_constraints:
    variable = constraint.lhs
    constraint_val = constraint.rhs
    constraint_type = constraint.sense
    print(f"{constraint}, Variable: {variable}, Constraint Value: {constraint_val}, constraint_type: {constraint_type}")
    constraint.rhs.add(-constraint_val + 100)
    print(f"NEW: {constraint}")
'''

'\nfor constraint in linear_constraints:\n    variable = constraint.lhs\n    constraint_val = constraint.rhs\n    constraint_type = constraint.sense\n    print(f"{constraint}, Variable: {variable}, Constraint Value: {constraint_val}, constraint_type: {constraint_type}")\n    constraint.rhs.add(-constraint_val + 100)\n    print(f"NEW: {constraint}")\n'

In [166]:
def find_ranges(minimal_explanation: list, epsilon: float):
    constraint_LE = None
    constraint_GE = None
    for i, constraint in enumerate(minimal_explanation):
        print('Initial constraint:' + '\t', constraint)

        variable = constraint.lhs
        constraint_val = constraint.rhs.constant
        #Check if its a (var <= value) constraint
        if constraint.sense == docplex.mp.constants.ComparisonType.LE: 
            #the value in(var <= value) is set to the maximum possible, i.e. the upper bound
            constraint.rhs = variable.ub
            print(f"LE constraint set to upper bound {variable.ub} --> {constraint}")
            
            testing_model.minimize(variable)            
            sol = testing_model.solve()
            if sol:
                print(f"Changed class with value = {testing_model.objective_value}")
                print(f"Diference between original value and solver found value: {abs(constraint_val) - abs(testing_model.objective_value)}")
                print(f"Epsilon >= Diference? {abs(abs(constraint_val) - abs(testing_model.objective_value)) <= epsilon}")
                if constraint_val <= (testing_model.objective_value - epsilon):
                    print(constraint_val, (testing_model.objective_value - epsilon))
                    constraint_LE = testing_model.objective_value - epsilon
                    
                else:
                    print("Entenred Validation Else for Min")
                    constraint_LE = constraint_val
                    
            else:
                print(f"Class not changed")
                constraint_LE = variable.ub
            constraint.rhs = constraint_val
            print(f"Reseted constraint to {constraint_val} --> {constraint} ")
            

        #Check if its a (var >= value)
        elif constraint.sense == docplex.mp.constants.ComparisonType.GE:
            constraint.rhs = variable.lb
            print(f"GE constraint set to lower bound {variable.lb} --> {constraint}")
            
            testing_model.maximize(variable)            
            sol = testing_model.solve()
            if sol:
                print(f"Changed class with value = {testing_model.objective_value}")
                print(f"Diference between original value and solver found value: {abs(constraint_val) - abs(testing_model.objective_value)}")
                print(f"Epsilon >= Diference? {abs(abs(constraint_val) - abs(testing_model.objective_value)) <= epsilon}")
                if constraint_val >= (testing_model.objective_value + epsilon):
                    constraint_GE = testing_model.objective_value + epsilon
                else:
                    print("Entered Validation Else for Max")
                    constraint_GE = constraint_val
            else:
                print(f"Class not changed")
                constraint_GE = variable.lb

            
            print(constraint_LE, variable.ub, constraint_LE == variable.ub)
            print(constraint_GE, variable.lb, constraint_GE == variable.lb)
            if (constraint_LE and constraint_GE) :
                
                if constraint_GE != variable.lb:
                    constraint.rhs = constraint_GE
                    
                else:
                    constraint.rhs = variable.lb
                    
                if constraint_LE != variable.ub:
                    minimal_explanation[i-1].rhs = constraint_LE
                else:
                    minimal_explanation[i-1].rhs = variable.ub
                    
                constraint_LE = None
                constraint_GE = None
                print(f"UPDATED Bounds for {variable}: {constraint.rhs.constant}, {minimal_explanation[i-1].rhs.constant}")
                print(minimal_explanation[i-1])
                print(constraint)
                
            else:

                print("FAILED")
        else:
            raise Exception('Constraint sense was neither LE nor GE')
        print("\n")
            
            

In [167]:
#find_bounds(linear_constraints)
find_ranges(linear_constraints, epsilon = 0.00001)

Initial constraint:	 input LE: x_0 <= 2.967691214515491
LE constraint set to upper bound 5.1279016612406805 --> input LE: x_0 <= 5.1279016612406805
Class not changed
Reseted constraint to 2.967691214515491 --> input LE: x_0 <= 2.967691214515491 


Initial constraint:	 input GE: x_0 >= 2.967691214515491
GE constraint set to lower bound -2.343651902203461 --> input GE: x_0 >= -2.343651902203461
Changed class with value = 1.4235358259063398
Diference between original value and solver found value: 1.5441553886091512
Epsilon >= Diference? False
5.1279016612406805 5.1279016612406805 True
1.4235458259063398 -2.343651902203461 False
UPDATED Bounds for x_0: 1.4235458259063398, 5.1279016612406805
input LE: x_0 <= 5.1279016612406805
input GE: x_0 >= 1.4235458259063398


Initial constraint:	 input LE: x_3 <= -1.408120229258977
LE constraint set to upper bound 4.136918742833424 --> input LE: x_3 <= 4.136918742833424
Changed class with value = -1.408115213286492
Diference between original value and 

In [149]:
testing_model.find_matching_linear_constraints('input')

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.1279016612406805),
 docplex.mp.LinearConstraint[input GE](x_0,GE,1.4235458259063398),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.408120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.790702170757714),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.24128294844203),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.24127975754059),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.6037614142464092),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [155]:
testing_model.minimize(1)
sol = testing_model.solve()
sol

docplex.mp.solution.SolveSolution(obj=1,values={x_0:1.42355,x_1:-3.4547,..

In [150]:
#print(testing_model.export_as_lp_string())

In [134]:
for i in range(129, 140):
    print((testing_model.get_var_by_index(i)), testing_model.get_var_by_index(i).value)

AttributeError: 'Var' object has no attribute 'value'

In [151]:
testing_model.get_var_by_index(129), testing_model.get_var_by_index(130), testing_model.get_var_by_index(131), testing_model.get_var_by_index(132), testing_model.get_var_by_index(133)

(docplex.mp.Var(type=C,name='o_0',lb=-4.02821,ub=5.56993),
 docplex.mp.Var(type=C,name='o_1',lb=-2.58205,ub=8.58841),
 docplex.mp.Var(type=C,name='o_2',lb=-9.76166,ub=5.40934),
 docplex.mp.Var(type=C,name='o_3',lb=-6.57575,ub=10.7016),
 docplex.mp.Var(type=C,name='o_4',lb=-8.34348,ub=13.5825))

In [168]:
for element in testing_model.solution.iter_var_values():
    print(element)

(docplex.mp.Var(type=C,name='x_0',lb=-2.34365,ub=5.1279), 1.4235458259063398)
(docplex.mp.Var(type=C,name='x_1',lb=-3.4547,ub=3.20854), -3.454697800980477)
(docplex.mp.Var(type=C,name='x_2',lb=-1.9278,ub=1.22542), -1.9277991250773103)
(docplex.mp.Var(type=C,name='x_3',lb=-2.33565,ub=4.13692), -1.408120229258977)
(docplex.mp.Var(type=C,name='x_4',lb=-3.75136,ub=3.40098), 2.322159623082899)
(docplex.mp.Var(type=C,name='x_5',lb=-0.790702,ub=8.67252), -0.790702170757714)
(docplex.mp.Var(type=C,name='x_6',lb=-2.47225,ub=5.10777), 4.24128294844203)
(docplex.mp.Var(type=C,name='x_7',lb=-0.361529,ub=5.87065), -0.3615292659832898)
(docplex.mp.Var(type=C,name='x_8',lb=-0.603761,ub=4.57028), -0.6037614142464092)
(docplex.mp.Var(type=C,name='y_0_1',ub=4.79992), 0.968703634902088)
(docplex.mp.Var(type=C,name='y_0_2',ub=9.89208), 2.0690504768469284)
(docplex.mp.Var(type=C,name='y_0_3',ub=5.75591), 2.173907716541208)
(docplex.mp.Var(type=C,name='y_0_5',ub=12.35), 0.6352030764746257)
(docplex.mp.Var(t

In [708]:
find_constraints(linear_constraints)

input LE: x_0 <= 5.1279016612406805
input GE: x_0 >= 1.4235458259063398


input LE: x_3 <= -1.408120229258977
input GE: x_3 >= -1.408120229258977


input LE: x_5 <= -0.790702170757714
input GE: x_5 >= -0.790702170757714


input LE: x_6 <= 4.24128294844203
input GE: x_6 >= 4.24127975754059


input LE: x_7 <= -0.3615292659832898
input GE: x_7 >= -0.3615292659832898


input LE: x_8 <= -0.6037614142464092
input GE: x_8 >= -0.6037614142464092




In [ ]:
log_and_improve_explanation(linear_constraints, epsilon=0.00001)

Initial constraint:	 input LE: x_0 <= 2.967691214515491


In [696]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.127691214515754),
 docplex.mp.LinearConstraint[input GE](x_0,GE,1.423691214515661),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.408120229258977),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.5191202292589647),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.790702170757714),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.24127975754059),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.24127975754059),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.6037614142464092),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

In [269]:
number_of_inputs = len(dataframe.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

In [270]:
improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.1279016612406805),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-2.3420983387592487),
 docplex.mp.LinearConstraint[input LE](x_0,LE,5.126348097796468),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-2.343651902203461),
 docplex.mp.LinearConstraint[input LE](x_3,LE,4.136918742833424),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-2.3330812571665267),
 docplex.mp.LinearConstraint[input LE](x_3,LE,4.134345979045517),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-2.335654020954433),
 docplex.mp.LinearConstraint[input LE](x_5,LE,8.672524288611543),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.7874757113883172),
 docplex.mp.LinearConstraint[input LE](x_5,LE,8.669297829242147),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,5.107769273886023),
 docplex.mp.LinearConstraint[input GE](x_6,GE,-2.4722307261139034),
 docplex.mp.LinearConstraint[input LE](x_6,LE,5.107747163416963)